In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Mon Sep 16 11:45:51 2019

@author: Sean
"""

import pandas as pd
import numpy as np
import matplotlib as mpl

def parse_year(date):
    
    date.strip()
    if len(date) == 4:
        year = date
    elif '/' in date:               
            tempYear = date[date.rfind('/')+1:]
            if len(tempYear) == 2:
                year = int('20'+tempYear)
                 
            else:
                year = tempYear
    elif '-' in date:
            year = int(date[0:4])            
    return pd.datetime(int(year),1,1)  

data = pd.read_csv('ViceNews_FullOISData - Sheet1.csv',delimiter = ',', parse_dates=['Date'],date_parser = parse_year)
data2 = pd.read_csv('denver_police_officer_involved_shootings.csv',delimiter=',')


ArmedAndFatal = np.sum(np.logical_and(data.Fatal == 'F',data.SubjectArmed == 'Y'))
ArmedAndFatal += np.sum(np.logical_and(data2.ARMED_WITH != 'None',data2.CASUALTY == 'Deceased'))
UnarmedAndFatal = np.sum(np.logical_and(data.Fatal == 'F',data.SubjectArmed == 'N'))
UnarmedAndFatal += np.sum(np.logical_and(data2.ARMED_WITH == 'None',data2.CASUALTY == 'Deceased'))
ArmedAndNonfatal = np.sum(np.logical_and(data.Fatal == 'N',data.SubjectArmed == 'Y'))
ArmedAndNonfatal += np.sum(np.logical_and(data2.ARMED_WITH != 'None',data2.CASUALTY != 'Deceased'))
UnarmedAndNonfatal = np.sum(np.logical_and(data.Fatal == 'N',data.SubjectArmed == 'N'))
UnarmedAndNonfatal += np.sum(np.logical_and(data2.ARMED_WITH == 'None',data2.CASUALTY != 'Deceased'))

x = np.zeros(7)
y1 = np.zeros(7)
y2 = np.zeros(7)
FatallyShotByYear = np.zeros(7)
NonfatallyShotByYear = np.zeros(7)
i=0
for year in range(2010,2017):
    FatallyShotByYear[i] = np.sum(np.logical_and(data.Date == pd.datetime(int(year),1,1), data.Fatal == 'F'))
    NonfatallyShotByYear[i] = np.sum(np.logical_and(data.Date == pd.datetime(int(year),1,1), data.Fatal == 'N'))
    x[i] = int(year)
    y1[i] = FatallyShotByYear[i]
    y2[i] = NonfatallyShotByYear[i]
    i = i+1

FatallyShotByRace = dict()
NonfatallyShotByRace = dict()
y3 = np.zeros(7)
y4 = np.zeros(7)
i=0
for race in ['W','L','B','U','A','O','NA']:
    FatallyShotByRace[race] = np.sum(np.logical_and(data.SubjectRace == race, data.Fatal == 'F'))
    FatallyShotByRace[race] += np.sum(np.logical_and(data2.RACE == race, data2.CASUALTY == 'Deceased'))
    NonfatallyShotByRace[race] = np.sum(np.logical_and(data.SubjectRace == race, data.Fatal == 'F'))
    NonfatallyShotByRace[race] += np.sum(np.logical_and(data2.RACE == race, data2.CASUALTY != 'Deceased'))
    y3[i] = float(FatallyShotByRace[race])
    y4[i] = float(NonfatallyShotByRace[race])
    i+=1    
    
FatallyShotByGender = np.zeros(3)
NonfatallyShotByGender = np.zeros(3)    

FatallyShotByGender[0] = np.sum(np.logical_and(data.Fatal == 'F',data.SubjectGender == 'M'))   
FatallyShotByGender[0] += np.sum(np.logical_and(data2.CASUALTY == 'Deceased',data2.GENDER == 'M'))   
FatallyShotByGender[1] = np.sum(np.logical_and(data.Fatal == 'F',data.SubjectGender == 'F')) 
FatallyShotByGender[1] += np.sum(np.logical_and(data2.CASUALTY == 'Deceased',data2.GENDER == 'F'))    
FatallyShotByGender[2] = np.sum(np.logical_and(data.Fatal == 'F',data.SubjectGender == 'U'))         
NonfatallyShotByGender[0] = np.sum(np.logical_and(data.Fatal == 'N',data.SubjectGender == 'M'))   
NonfatallyShotByGender[0] += np.sum(np.logical_and(data2.CASUALTY != 'Deceased',data2.GENDER == 'M'))  
NonfatallyShotByGender[1] = np.sum(np.logical_and(data.Fatal == 'N',data.SubjectGender == 'F'))
NonfatallyShotByGender[1] += np.sum(np.logical_and(data2.CASUALTY != 'Deceased',data2.GENDER == 'F'))   
NonfatallyShotByGender[2] = np.sum(np.logical_and(data.Fatal == 'N',data.SubjectGender == 'U')) 

mpl_fig1 = mpl.pyplot.figure()
ax = mpl_fig1.add_subplot(111)                
label = ['Armed,Fatally shot','Armed,Nonfatally shot','Unarmed,Fatally shot','Unarmed,Nonfatally shot']                
p1 = ax.bar(np.arange(2),[ArmedAndFatal,UnarmedAndFatal],0.5,color=(0,0,0))
p2 = ax.bar(np.arange(2),[ArmedAndNonfatal,UnarmedAndNonfatal],0.5,color=(0.5,0.5,0.5),bottom=[ArmedAndFatal,UnarmedAndFatal])
ax.set_ylabel('Subjects')
ax.set_xlabel('Status')
ax.set_title('Comparison of armed and unarmed OIS subjects')       
ax.set_xticks([0,1])
ax.set_xticklabels(('Armed','Unarmed'))   
ax.legend(['Fatally shot','Nonfatally shot'])

mpl_fig2 = mpl.pyplot.figure()
ax = mpl_fig2.add_subplot(111)
p1 = ax.plot(x,y1)
p2 = ax.plot(x,y2)  
ax.set_ylabel('Shootings')
ax.set_xlabel('Year')
ax.set_title('Survival of officer-involved shootings has declined annually')
ax.legend(['Fatally shot','Survived'])

mpl_fig3 = mpl.pyplot.figure()
ax = mpl_fig3.add_subplot(111)
p1 = ax.bar(x,y3,0.5,color=(0,0,0))
p2 = ax.bar(x,y4,0.5,color=(0.5,0.5,0.5),bottom = y3)
ax.set_ylabel('Shootings')
ax.set_xlabel('Subject Race')
ax.set_title('Shootings by Subject Race')
ax.set_xticks(x)
ax.set_xticklabels(FatallyShotByRace.keys())
ax.legend(['Fatally shot','Survived'])

mpl_fig4 = mpl.pyplot.figure()
ax = mpl_fig4.add_subplot(111)
p1 = ax.bar([0,1,2],FatallyShotByGender,0.5,color=(0,0,0))
p2 = ax.bar([0,1,2],NonfatallyShotByGender,0.5,color=(0.5,0.5,0.5),bottom=FatallyShotByGender)
ax.set_ylabel('Shootings')
ax.set_xlabel('Subject Gender')
ax.set_title('Shootings by Subject Gender')
ax.set_xticks([0,1,2])
ax.set_xticklabels(['Male','Female','Unknown'])
ax.legend(['Fatally shot','Survived'])
